In [1]:
##########################################################################
##### Template code and instructions to scrape Rate My Professor Comments#
##########################################################################
### Step 1: install packages - 
##### packages are necessary to install and load, given that they have the built in functions necessary to run complex tasks. 
## They effectively act as one of the most crucial time saving activities that would otherwise lead to overly long and 
## duplicative scripts. 

## install pkgs 
import sys
!{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install requests #; this code here can be used to install packages on anaconda/jupyter notebook 
### I believe the below should be installed by default 
import requests # web scraping 
from bs4 import BeautifulSoup # for web scraping 
import itertools # for efficient operation of loops 
import pandas as pd # necessary for reading in, creating, and manipulating data frames 
import csv ## for importing/exporting csvs 
import glob ## for finding files in path
import re


In [2]:
### Step 2: Read in the csv with the data of interest 
## note: This will be the section that you update manually the most. The rest should be automated 

faculty_csv = pd.read_csv('faculty characteristics web scraping_UNC.csv')
urls = faculty_csv['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv['college']

##prof last name 
prof_lastname = faculty_csv['prof_lastname']

# prof first name 
prof_firstname = faculty_csv['prof_firstname']

## note: you will want to manually seach the rate my professor website for the professor of interest. From there, you will 
# be able to grab the url for the professor of interest, which will be the last numeric digits of the url that will vary. 
#url
faculty_csv

,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,notes
0,UNIVERSITY OF NORTH CAROLINA,DALE,WHITTINGTON,ENVIRONMENTAL SCIENCES & ENGINEERING,1,0,https://www.ratemyprofessors.com/professor/100...,NaN
1,UNIVERSITY OF NORTH CAROLINA,COURTNEY,WOODS,ENVIRONMENTAL SCIENCES & ENGINEERING,0,1,https://www.ratemyprofessors.com/professor/225...,NaN
2,UNIVERSITY OF NORTH CAROLINA,KARI,NORTH,EPIDEMIOLOGY,0,0,https://www.ratemyprofessors.com/professor/212...,NaN
3,UNIVERSITY OF NORTH CAROLINA,SUZANNE,MAMAN,HEALTH BEHAVIOR,0,0,https://www.ratemyprofessors.com/professor/286...,Ratings are for Health Science
4,UNIVERSITY OF NORTH CAROLINA,BENJAMIN,MEIER,HEALTH POLICY & MANAGEMENT,1,0,https://www.ratemyprofessors.com/professor/143...,Ratings are for Public Policy
...,...,...,...,...,...,...,...,...
97,UNIVERSITY OF NORTH CAROLINA,TED,MOUW,SOCIOLOGY,1,0,https://www.ratemyprofessors.com/professor/379594,NaN
98,UNIVERSITY OF NORTH CAROLINA,LISA,PEARCE,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/221...,NaN
99,UNIVERSITY OF NORTH CAROLINA,ALEXANDREA,RAVENELLE,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/259...,NaN
100,UNIVERSITY OF NORTH CAROLINA,JESSICA,SU,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/252...,NaN


In [3]:
### Step 3: run nested loop with these data

for j, element in enumerate(urls):
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    ## split up into difficulty, quality and class num 
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        d1.append({
            'row': i+1,
            'quality_of_class': prof_qual[i].get_text(),
            "difficulty_of_class": prof_dif[i].get_text(),
            "class_code": prof_class[i].get_text(),
            "college": college[j], #jth lvl, not ith level, variable 
            "prof_firstname": prof_firstname[j],            
            "prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
            'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ### now save the file 
    save_name = "scraped_data/rmp_data" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)







In [4]:
#### compile all of the files in the directory ## 
scraped_data_files =  glob.glob("scraped_data/*")
scraped_data_files_unc = [x for x in scraped_data_files if re.findall("NORTH CAROLINA",x)] 
# grab files that have "zest" in title
scraped_data_files_unc


['scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ABRAMOWITZ_JONATHAN.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ALDRICH_HOWARD.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ALGOE_SARA.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ANDERSON_ASHLEY.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ANDREWS_KENNETH.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ARNOLD_JENNIFER.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BALLARD-ROSA_CAMERON.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BAPAT_NAVIN.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BARDONE-CONE_ANNA.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BASSI_ANNA.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BAUCOM_DONALD.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BAUER_DANIEL.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BAUMGARTNER_FRANK.csv',
 'scraped_data\\rmp

In [5]:
master_unc = pd.DataFrame()
  
for element in scraped_data_files_unc:
    temp_df = pd.read_csv(element)
    master_unc = master_unc.append(temp_df)

    
           
        
    
    
    
    
    

C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4287820039.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4287820039.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4287820039.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4287820039.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\Ap

EmptyDataError: No columns to parse from file

In [6]:
master_unc

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment
0,1,5.0,1.0,PSYC89H,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,TAKE THIS CLASS as a freshman. It's an easy an...
1,2,5.0,1.0,PSYC89H,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,"If you are a freshman, TAKE THIS CLASS. It is ..."
2,3,5.0,3.0,PSYC245,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,TAKE THIS CLASS: super interesting and great l...
3,4,5.0,1.0,PSYC245,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,Abramowitz is the best! So knowledgable about ...
4,5,5.0,1.0,PSYC245,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,If you need an easy A and don't mind attending...
...,...,...,...,...,...,...,...,...
15,16,4.0,3.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,"if you thought Ap Stat was easy, this is basic..."
16,17,3.0,4.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,Dr. Sathy is a very nice person in the classro...
17,18,4.0,4.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,"Dr. Sathy is very kind, and it's obvious that ..."
18,19,5.0,1.0,PSYC89,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,Kinda a joke. Had to get laser cutter certifie...


In [7]:
#### now create the empty columns
### outrage vars 
master_unc["out_misrep"] = 0
master_unc["out_emo_lang"] = 0
master_unc["out_ideo_extrem"] = 0
master_unc["out_slip_slope"] = 0
master_unc["out_insult"] = 0
master_unc["out_mock_sarc"] = 0
master_unc["out_threat"] = 0
## personal attacks 
master_unc["pa_dress"] = 0
master_unc["pa_appear"] = 0
master_unc["pa_attit"] = 0
master_unc["pa_interact"] = 0
master_unc["pa_humor"] = 0
master_unc["pa_other"] = 0
### predjudice and bigotry 
master_unc["pb_sex"] = 0
master_unc["pb_lgbtq"] = 0
master_unc["pb_race"] = 0
master_unc["pb_origin"] = 0
master_unc["pb_nuero_div"] = 0
master_unc["pb_phys_able"] = 0
master_unc["pb_pol_affil"] = 0
### the comments that might be helpful 
master_unc["complex"] = 0
master_unc["constructive"] = 0
master_unc["reflective"] = 0


### print the data frame here; should have more than 8 cols 
master_unc






,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_sex,pb_lgbtq,pb_race,pb_origin,pb_nuero_div,pb_phys_able,pb_pol_affil,complex,constructive,reflective
0,1,5.0,1.0,PSYC89H,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,TAKE THIS CLASS as a freshman. It's an easy an...,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,5.0,1.0,PSYC89H,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,"If you are a freshman, TAKE THIS CLASS. It is ...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,5.0,3.0,PSYC245,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,TAKE THIS CLASS: super interesting and great l...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,5.0,1.0,PSYC245,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,Abramowitz is the best! So knowledgable about ...,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,5.0,1.0,PSYC245,UNIVERSITY OF NORTH CAROLINA,JONATHAN,ABRAMOWITZ,If you need an easy A and don't mind attending...,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,16,4.0,3.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,"if you thought Ap Stat was easy, this is basic...",0,0,...,0,0,0,0,0,0,0,0,0,0
16,17,3.0,4.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,Dr. Sathy is a very nice person in the classro...,0,0,...,0,0,0,0,0,0,0,0,0,0
17,18,4.0,4.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,"Dr. Sathy is very kind, and it's obvious that ...",0,0,...,0,0,0,0,0,0,0,0,0,0
18,19,5.0,1.0,PSYC89,UNIVERSITY OF NORTH CAROLINA,VIJI,SATHY,Kinda a joke. Had to get laser cutter certifie...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
### now export 
master_unc.to_csv("master_unc_data_comments.csv", index=False)


In [44]:
#### Import the OSU data 
faculty_csv = pd.read_csv('faculty characteristics web scraping_OSU2.csv')

faculty_csv=faculty_csv.loc[faculty_csv['link_rmp'] != "DNE"]

### lets reindex 
#j_range = list(range(0,len(faculty_csv),1))
#print(j_range)

urls = faculty_csv['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv['college']

##prof last name 
prof_lastname = faculty_csv['prof_lastname']

# prof first name 
prof_firstname = faculty_csv['prof_firstname']

## note: you will want to manually seach the rate my professor website for the professor of interest. From there, you will 
# be able to grab the url for the professor of interest, which will be the last numeric digits of the url that will vary. 
#url
faculty_csv




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,NOTES
0,OHIO STATE UNIVERSITY,NAOMI,ADANIYA,PUBLIC HEALTH,0,1,https://www.ratemyprofessors.com/professor/207...,NaN
1,OHIO STATE UNIVERSITY,SARA,ANDERSON,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/272...,NaN
2,OHIO STATE UNIVERSITY,REBECCA,ANDRIDGE,PUBLIC HEALTH,0,1,https://www.ratemyprofessors.com/professor/240...,NaN
3,OHIO STATE UNIVERSITY,KELLIE,ARCHER,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/245...,NaN
4,OHIO STATE UNIVERSITY,GIA,BARBOZA-SALERNO,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/287...,NaN
...,...,...,...,...,...,...,...,...
225,OHIO STATE UNIVERSITY,JOHN,MUELLER,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/175682,NaN
226,OHIO STATE UNIVERSITY,RANDALL,RIPLEY,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/811378,NaN
227,OHIO STATE UNIVERSITY,GOLDIE,SHABAD,POLITICAL SCIENCE,0,0,https://www.ratemyprofessors.com/professor/93027,NaN
228,OHIO STATE UNIVERSITY,ELLIOT,SLOTNICK,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/51211,NaN


In [16]:
len(urls)

230

In [17]:
### Step 3: run nested loop with these data

for j, element in enumerate(urls):
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    ## split up into difficulty, quality and class num 
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        d1.append({
            'row': i+1,
            'quality_of_class': prof_qual[i].get_text(),
            "difficulty_of_class": prof_dif[i].get_text(),
            "class_code": prof_class[i].get_text(),
            "college": college[j], #jth lvl, not ith level, variable 
            "prof_firstname": prof_firstname[j],            
            "prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
            'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ### now save the file 
    save_name = "scraped_data/rmp_data" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)







In [18]:
### Now let's grab only the OSU files 

#### compile all of the files in the directory ## 
scraped_data_files =  glob.glob("scraped_data/*")
scraped_data_files_osu = [x for x in scraped_data_files if re.findall("OHIO STATE",x)] # grab files that have "zest" in title
scraped_data_files_osu

['scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ACS_ALEX.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ADANIYA_NAOMI.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ANDERSON_SARA.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ANDRIDGE_REBECCA.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ANGEL_WILLIAM.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ARCHER_KELLIE.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ARKES_HAL.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ASHER_HERB.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BAKER_JEREMY.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BARBOZA-SALERNO_GIA.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BASTA_NICHOLAS.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BAUM_LAWRENCE.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BECK_PAUL.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BECK_STEVEN.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BELLAIR_PAUL.csv',
 'scr

In [20]:
### now loop in and read the data 
master_osu = pd.DataFrame()

## note: it appears that there are some csvs that simply have no data (i.e. Box Steffensmier.) We will therefore want to go 
#and discard those profs that are characterized by these, and then proceed to skip them, though make note of them. 
## see the use of the try command for this purpose 
  
for element in scraped_data_files_osu:
    try:
        temp_df = pd.read_csv(element)
        temp_len = len(temp_df)
        master_osu = master_osu.append(temp_df)
    except: ## passes error, though notes it 
        pass
        print("The evals for ", element, "not appended; had length of", temp_len)

#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_BOX-STEFFENSMEIER_JANET.csv not appended; had length of 11
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_GUNTHER_RICHARD.csv not appended; had length of 15
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_HUFTON_AMIE.csv not appended; had length of 20
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_IBARAKI_MOTOMU.csv not appended; had length of 19
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_KRAJBICH_IAN.csv not appended; had length of 10
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_MCGRAW_KATHLEEN.csv not appended; had length of 2
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_PATSON_NIKOLE.csv not appended; had length of 20
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_REMPALA_GRZEGORZ.csv not appended; had length of 6
    
           
        
    
    
    
    
    

C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curie

The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_BOX-STEFFENSMEIER_JANET.csv not appended; had length of 11
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_GUNTHER_RICHARD.csv not appended; had length of 15
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_HUFTON_AMIE.csv not appended; had length of 20
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_IBARAKI_MOTOMU.csv not appended; had length of 19
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_KRAJBICH_IAN.csv not appended; had length of 10
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_MCGRAW_KATHLEEN.csv not appended; had length of 2
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_PATSON_NIKOLE.csv not appended; had length of 20
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_REMPALA_GRZEGORZ.csv not appended; had length of 6


C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\3815159244.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curie

In [58]:
### We will want to specially scrape these profs again 

### create a list object 
missing_list = ['GUNTHER','MCGRAW','KRAJBICH']
#missing_list = ['GUNTHER','HUFTON','KRAJBICH','MCGRAW','PATSON','REMPALA']

## MOTOMU IBARAKI is missing viable url, and so is BOX-STEFFENSMEIER ; so is Amie Hufton ; so is Grzegorz Rempala
# so is PATSOn, 
## last two should be good 

## subset the faculty chr list 
faculty_csv_miss = faculty_csv[faculty_csv['prof_lastname'].isin(missing_list)]

#j_range = list(range(0,len(faculty_csv_miss),1))

#faculty_csv_miss.set_index(j_range)
faculty_csv_miss=faculty_csv_miss.reset_index()

#df[df['dog_name'].isin({'Fido', 'Yeller'})]
faculty_csv_miss #looks like it worked 


,index,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,NOTES
0,103,OHIO STATE UNIVERSITY,IAN,KRAJBICH,PSYCHOLOGY,1,0,https://www.ratemyprofessors.com/professor/227...,NaN
1,221,OHIO STATE UNIVERSITY,RICHARD,GUNTHER,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/175034,NaN
2,224,OHIO STATE UNIVERSITY,KATHLEEN,MCGRAW,POLITICAL SCIENCE,0,99,https://www.ratemyprofessors.com/professor/175678,NaN


In [59]:
### format var list 
urls = faculty_csv_miss['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv_miss['college']

##prof last name 
prof_lastname = faculty_csv_miss['prof_lastname']

# prof first name 
prof_firstname = faculty_csv_miss['prof_firstname']

### need to reindex 


In [60]:
prof_lastname[0] # if this pulls up gunther, then it works 

'KRAJBICH'

In [61]:
### loop for the missing profs 
for j, element in enumerate(urls):
    print(j)
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    ## split up into difficulty, quality and class num 
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        d1.append({
            'row': i+1,
            'quality_of_class': prof_qual[i].get_text(),
            "difficulty_of_class": prof_dif[i].get_text(),
            "class_code": prof_class[i].get_text(),
            #"college": college[j], #jth lvl, not ith level, variable 
            #"prof_firstname": prof_firstname[j],            
            #"prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
            'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ## add j level vars here
    df['college'] = college[j]
    df['prof_firstname'] = prof_firstname[j]
    df['prof_lastname'] = prof_lastname[j]
    
    ### now save the file 
    save_name = "scraped_data/rmp_data" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)

0
1
2


In [62]:
## we now should be good; let's redo 
scraped_data_files =  glob.glob("scraped_data/*")
scraped_data_files_osu = [x for x in scraped_data_files if re.findall("OHIO STATE",x)] # grab files that have "zest" in title

## now the append loop 
master_osu = pd.DataFrame()

## note: it appears that there are some csvs that simply have no data (i.e. Box Steffensmier.) We will therefore want to go 
#and discard those profs that are characterized by these, and then proceed to skip them, though make note of them. 
## see the use of the try command for this purpose 
  
for element in scraped_data_files_osu:
    try:
        temp_df = pd.read_csv(element)
        temp_len = len(temp_df)
        master_osu = master_osu.append(temp_df)
    except: ## passes error, though notes it 
        pass
        print("The evals for ", element, "not appended; had length of", temp_len)


C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curie

The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_BOX-STEFFENSMEIER_JANET.csv not appended; had length of 11
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_HUFTON_AMIE.csv not appended; had length of 20
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_IBARAKI_MOTOMU.csv not appended; had length of 19
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_PATSON_NIKOLE.csv not appended; had length of 20
The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_REMPALA_GRZEGORZ.csv not appended; had length of 6


C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_28932\4023397035.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curie

In [63]:
#### now create the empty columns
### outrage vars 
master_osu["out_misrep"] = 0
master_osu["out_emo_lang"] = 0
master_osu["out_ideo_extrem"] = 0
master_osu["out_slip_slope"] = 0
master_osu["out_insult"] = 0
master_osu["out_mock_sarc"] = 0
master_osu["out_threat"] = 0
## personal attacks 
master_osu["pa_dress"] = 0
master_osu["pa_appear"] = 0
master_osu["pa_attit"] = 0
master_osu["pa_interact"] = 0
master_osu["pa_humor"] = 0
master_osu["pa_other"] = 0
### predjudice and bigotry 
master_osu["pb_sex"] = 0
master_osu["pb_lgbtq"] = 0
master_osu["pb_race"] = 0
master_osu["pb_origin"] = 0
master_osu["pb_nuero_div"] = 0
master_osu["pb_phys_able"] = 0
master_osu["pb_pol_affil"] = 0
### the comments that might be helpful 
master_osu["complex"] = 0
master_osu["constructive"] = 0
master_osu["reflective"] = 0









In [64]:
master_osu.to_csv("master_osu_data_comments.csv", index=False)
